# Phase 3 - Function Calling

## About this phase
In this challenge you have to make sure, that your API is able to answer questions, that neither the model knows, nor are the answers hidden in your custom data. But you can make your model aware of a bunch of APIs that can be used to call the APIs. 
So go, make the model aware of the APIs and build an application that call the APIs whenever required!


If not already done run this in the top level folder:
```
pip install -r requirements.txt
```




In [ ]:
import json
import os
from enum import Enum
from pydantic import BaseModel
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("OPENAI_API_VERSION")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION")
)
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

This is the object model for receiving questions.

In [ ]:

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_or_false = "true_or_false"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


## Sample Function 

In [ ]:
import requests

smoorghApi = "https://.northeurope.azurecontainerapps.io/"

def get_movie_rating(title):
    try:
        response = requests.get(f"{smoorghApi}/rating/{title}")
        print('The api response for rating is:', response.text)
        return response.text
    
    except:
        return "Sorry, I couldn't find a rating for that movie."

def get_movie_year(title):
    try:
        response = requests.get(f"{smoorghApi}/year/{title}")
        print('The api response for year is:', response.text)
        return response.text

    except:
        return "Sorry, I couldn't find a year for that movie."
    
def get_movie_actor(title):
    try:
        response = requests.get(f"{smoorghApi}/actor/{title}")
        print('The api response for actor is:', response.text)
        return response.text

    except:
        return "Sorry, I couldn't find an actor for that movie."
    
def get_movie_location(title):
    try:
        response = requests.get(f"{smoorghApi}/location/{title}")
        print('The api response for location is:', response.text)
        return response.text

    except:
        return "Sorry, I couldn't find a location for that movie."

def get_movie_genre(title):
    try:
        response = requests.get(f"{smoorghApi}/genre/{title}")
        print('The api response for genre is:', response.text)
        return response.text

    except:
        return "Sorry, I couldn't find a genre for that movie."

print(get_movie_rating("The Lost Planet"))

print(get_movie_year("The Lost Planet"))

print(get_movie_actor("The Lost Planet"))

print(get_movie_location("The Lost Planet"))

print(get_movie_genre("The Lost Planet"))


Make the model aware of the function


In [ ]:
functions = [
        {
            "type": "function",
            "function": {
                "name": "get_movie_rating",
                "description": "Gets the rating of a movie",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "The movie name. The movie name should be a string without quotation marks.",
                        }
                    },
                    "required": ["title"],
                },
            }
        },
         {
            "type": "function",
            "function": {
                "name": "get_movie_location",
                "description": "Gets the location of a movie",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "The movie name. The movie name should be a string without quotation marks.",
                        }
                    },
                    "required": ["title"],
                },
            }
        }
    ]   
available_functions = {
            "get_movie_rating": get_movie_rating,
            "get_movie_location": get_movie_location        
        } 

# YOUR Mission: 
Adjust the function below and reuse it in the main.py file later to deploy to Azure and to update your service. 
Ensure the answers provided are correct and in the correct format.


In [ ]:

async def ask_question(ask: Ask):
    """
    Ask a question
    """
    
    question = ask.question
    messages= [{"role" : "assistant", "content" : question}
               , { "role" : "system", "content" : "Answer the question of the user and use the tools available to you."}
               ]
    first_response = client.chat.completions.create(
        model = deployment_name,
        messages = messages,
        tools = functions,
        tool_choice = "auto",
    )

    print(first_response)
    response_message = first_response.choices[0].message
    tool_calls = response_message.tool_calls

     # Step 2: check if GPT wanted to call a function
    if tool_calls:
        print("Recommended Function call:")
        print(tool_calls)
        print()
    
        # Step 3: call the function
        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            # verify function exists
            if function_name not in available_functions:
                return "Function " + function_name + " does not exist"
            else:
                print("Calling function: " + function_name)
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(function_args)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            ) 
            print("Addding this message to the next prompt:") 
            print (messages)
            
             # extend conversation with function response
            second_response = client.chat.completions.create(
                model = model_name,
                messages = messages)  # get a new response from the model where it can see the function response
            
            print("second_response")
            
            return second_response.choices[0].message.content
            

   

Use this snippet to try your method with several questions.

In [ ]:

ask = Ask(question="Which country does 'Hobbiton: The Heist of the Silver Dragon' take place in?", type=QuestionType.estimation)
answer = await ask_question(ask)
print('Answer:', answer )


Make sure you transfer your code changes into main.py (or additional files). Then redeploy your container using this command.
```
bash ./azd-hooks/deploy.sh phase3 $AZURE_ENV_NAME
```
Make sure to provide the URL of your endpoint in the team portal!